In [31]:
from copy import deepcopy

In [32]:
"""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

REGS_TEST = {"A": 729, "B": 0, "C": 0}
PROG_TEST = [int(i) for i in "0,1,5,4,3,0".split(',')]

"""Register A: 123729
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""
REGS_TEST_1 = {"A": 123729, "B": 0, "C": 0}
PROG_TEST_1 = [int(i) for i in "0,3,5,4,3,0".split(',')]

"""Register A: 64012472
Register B: 0
Register C: 0

Program: 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0"""
REGS = {"A": 64012472, "B": 0, "C": 0}
PROG = [int(i) for i in "2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0".split(',')]


In [33]:
# There are two types of operands; each instruction specifies the type of its operand. 
# The value of a literal operand is the operand itself. 
# For example, the value of the literal operand 7 is the number 7. 
# The value of a combo operand can be found as follows:

# Combo operands 0 through 3 represent literal values 0 through 3.
# Combo operand 4 represents the value of register A.
# Combo operand 5 represents the value of register B.
# Combo operand 6 represents the value of register C.
# Combo operand 7 is reserved and will not appear in valid programs.
def combo(o, regs):
    if 0 <= o <= 3:
        return o
    elif o == 4:
        return regs['A']
    elif o == 5:
        return regs['B']
    elif o == 6:
        return regs['C']
    else:
        raise ValueError("invalid or reserved")

# The adv instruction (opcode 0) performs division. The numerator is the value in the A register. 
# The denominator is found by raising 2 to the power of the instruction's combo operand. 
# (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) 
# The result of the division operation is truncated to an integer and then written to the A register.
def adv(o, regs, pt, state):
    regs['A'] = regs['A'] >> combo(o, regs)
    pt += 2
    return regs, pt, state

# The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the 
# instruction's literal operand, then stores the result in register B.
def bxl(o, regs, pt, state):
    regs['B'] = regs['B'] ^ o
    pt += 2
    return regs, pt, state

# The bst instruction (opcode 2) calculates the value of its combo operand modulo 8 
# (thereby keeping only its lowest 3 bits), then writes that value to the B register.
def bst(o, regs, pt, state):
    regs['B'] = combo(o, regs) % 8
    pt += 2
    return regs, pt, state

# The jnz instruction (opcode 3) does nothing if the A register is 0. However, if the A register 
# is not zero, it jumps by setting the instruction pointer to the value of its literal operand; 
# if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.
def jnz(o, regs, pt, state):
    if regs['A'] == 0:
        pt += 2
        return regs, pt, state
    else:
        pt = o
        return regs, pt, state

# The bxc instruction (opcode 4) calculates the bitwise XOR of 
# register B and register C, then stores the result in register B.
# (For legacy reasons, this instruction reads an operand but ignores it.)
def bxc(o, regs, pt, state):
    regs['B'] = regs['B'] ^ regs['C']
    pt += 2
    return regs, pt, state

# The out instruction (opcode 5) calculates the value of its combo operand modulo 8, 
# then outputs that value. (If a program outputs multiple values, they are separated by commas.)
def out(o, regs, pt, state):
    pt += 2
    state += f"{combo(o, regs) % 8}"
    return regs, pt, state

# The bdv instruction (opcode 6) works exactly like the adv instruction except
# that the result is stored in the B register. 
# (The numerator is still read from the A register.)
def bdv(o, regs, pt, state):
    regs['B'] = regs['A'] >> combo(o, regs)
    pt += 2
    return regs, pt, state

# The cdv instruction (opcode 7) works exactly like the adv instruction except 
# that the result is stored in the C register. 
# (The numerator is still read from the A register.)
def cdv(o, regs, pt, state):
    regs['C'] = regs['A'] >> combo(o, regs)
    pt += 2
    return regs, pt, state

INS = {0: adv, 1: bxl, 2: bst, 3: jnz, 4: bxc, 5: out, 6: bdv, 7: cdv}


Part 1

In [34]:
# using mutable dicts
regs = deepcopy(REGS)
prog = PROG
state = ""
pt = 0

while pt < len(prog)-1:
    oc = prog[pt]
    o = prog[pt+1]
    regs, pt, state = INS[oc](o, regs, pt, state)

",".join(list(state))

'1,0,2,0,5,7,2,1,3'

Part 2

In [ ]:
# useful for checking a few values

prog = PROG
orig_state = "".join([str(s) for s in prog])

for i in range(100000):
    regs = deepcopy(REGS)
    regs['A'] = i
    state = ""
    pt = 0
    while pt < len(prog)-1:
        oc = prog[pt]
        o = prog[pt+1]
        regs, pt, state = INS[oc](o, regs, pt, state)
    print(i, oct(i), state)

In [36]:
def build(i, i_max, prog, orig_str, poss={0: [""]}):
    if i == i_max:
        return
    poss[i+1] = []
    for j in range(8):
        for s in poss[i]:
            rA_str = s + str(j)
            regs = {"A": int(rA_str, 8), "B": 0, "C": 0}
            # regs = (int(rA_str, 8), 0, 0)
            state = ""
            pt = 0
            while pt < len(prog)-1:
                oc = prog[pt]
                o = prog[pt+1]
                regs, pt, state = INS[oc](o, regs, pt, state)
            if state == orig_str[-1-i:]:
                poss[i+1].append(rA_str)
    build(i+1, i_max, prog, orig_str, poss)

In [37]:
prog = PROG
orig_state = "".join([str(s) for s in prog])
poss = {0: [""]}

build(0, 16, prog, orig_state, poss)

min_rA = min([int(x, 8) for x in poss[16]])
min_rA

265652340990875

In [38]:
# let's check that it works
# needs to pring the following program:
# 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0

prog = PROG
regs = {"A": min_rA, "B": 0, "C": 0}
state = ""
pt = 0
while pt < len(prog)-1:
    oc = prog[pt]
    o = prog[pt+1]
    regs, pt, state = INS[oc](o, regs, pt, state)
",".join(list(state))

'2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0'